In [1]:
import torch

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device 

/home/logan/Desktop/Youtube/Gradio/Text-Classification/project-code/venv/lib/python3.9/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


'cpu'

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [4]:
MODEL_PATH = 'roberta-base-go_emotions/'
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH)
model = model.to(device)

In [5]:
query = 'I am not having a great day.'

In [6]:
inputs = tokenizer(query, return_tensors='pt', truncation=True)
inputs = inputs.to(device)
inputs

{'input_ids': tensor([[  0, 100, 524,  45, 519,  10, 372, 183,   4,   2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [7]:
outputs = model(**inputs)

In [8]:
label2id = model.config.label2id
label2id

{'admiration': 0,
 'amusement': 1,
 'anger': 2,
 'annoyance': 3,
 'approval': 4,
 'caring': 5,
 'confusion': 6,
 'curiosity': 7,
 'desire': 8,
 'disappointment': 9,
 'disapproval': 10,
 'disgust': 11,
 'embarrassment': 12,
 'excitement': 13,
 'fear': 14,
 'gratitude': 15,
 'grief': 16,
 'joy': 17,
 'love': 18,
 'nervousness': 19,
 'neutral': 27,
 'optimism': 20,
 'pride': 21,
 'realization': 22,
 'relief': 23,
 'remorse': 24,
 'sadness': 25,
 'surprise': 26}

In [9]:
outputs[0]

tensor([[-5.8912, -6.4097, -5.1344, -2.3781, -4.4519, -5.2010, -5.6352, -6.0904,
         -5.8536,  0.0178, -2.8869, -5.3134, -5.1938, -5.9966, -5.9365, -7.3348,
         -5.6248, -5.0768, -5.9956, -4.2687, -5.9012, -7.4587, -4.2042, -5.7734,
         -5.2262, -0.6325, -6.4586, -2.8483]], grad_fn=<AddmmBackward0>)

In [10]:
logits = outputs.logits
sigmoid = torch.nn.Sigmoid()
probs = sigmoid(logits.squeeze().cpu())
probs = probs.detach().numpy()

In [11]:
probs

array([0.0027561 , 0.00164284, 0.00585601, 0.08485784, 0.01152237,
       0.00548085, 0.00355731, 0.00225947, 0.00286127, 0.50444835,
       0.05280709, 0.00490111, 0.00552022, 0.00248115, 0.00263443,
       0.000652  , 0.00359444, 0.00620087, 0.00248361, 0.01380647,
       0.00272877, 0.0005761 , 0.01471279, 0.00309946, 0.00534504,
       0.34694305, 0.00156451, 0.05476679], dtype=float32)

In [12]:
for i, k in enumerate(label2id.keys()):
    label2id[k] = probs[i]

In [13]:
label2id

{'admiration': 0.002756104,
 'amusement': 0.0016428408,
 'anger': 0.005856009,
 'annoyance': 0.08485784,
 'approval': 0.011522369,
 'caring': 0.0054808543,
 'confusion': 0.0035573125,
 'curiosity': 0.0022594687,
 'desire': 0.0028612658,
 'disappointment': 0.50444835,
 'disapproval': 0.052807093,
 'disgust': 0.004901111,
 'embarrassment': 0.0055202236,
 'excitement': 0.002481147,
 'fear': 0.00263443,
 'gratitude': 0.0006519979,
 'grief': 0.003594436,
 'joy': 0.006200869,
 'love': 0.0024836052,
 'nervousness': 0.013806469,
 'neutral': 0.0027287677,
 'optimism': 0.0005760981,
 'pride': 0.014712791,
 'realization': 0.0030994597,
 'relief': 0.0053450353,
 'remorse': 0.34694305,
 'sadness': 0.001564505,
 'surprise': 0.05476679}

In [14]:
label2id = {k: v for k, v in sorted(label2id.items(), key=lambda item: item[1], reverse=True)}
label2id

{'disappointment': 0.50444835,
 'remorse': 0.34694305,
 'annoyance': 0.08485784,
 'surprise': 0.05476679,
 'disapproval': 0.052807093,
 'pride': 0.014712791,
 'nervousness': 0.013806469,
 'approval': 0.011522369,
 'joy': 0.006200869,
 'anger': 0.005856009,
 'embarrassment': 0.0055202236,
 'caring': 0.0054808543,
 'relief': 0.0053450353,
 'disgust': 0.004901111,
 'grief': 0.003594436,
 'confusion': 0.0035573125,
 'realization': 0.0030994597,
 'desire': 0.0028612658,
 'admiration': 0.002756104,
 'neutral': 0.0027287677,
 'fear': 0.00263443,
 'love': 0.0024836052,
 'excitement': 0.002481147,
 'curiosity': 0.0022594687,
 'amusement': 0.0016428408,
 'sadness': 0.001564505,
 'gratitude': 0.0006519979,
 'optimism': 0.0005760981}